import pandas as pd
import matplotlib.pyplot as plt
import bokeh
import datetime as dt
from sklearn.manifold import TSNE
import plotly as py
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from dask import dataframe as dd
from sklearn.preprocessing import OneHotEncoder
import dash
from pathlib import Path
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
# ^^^ pyforest auto-imports - don't write above this line
## ✨❗ Network_Anomaly_Detection ✨❗


* Anomaly detection has been the main focus of many researchers’ due to its potential in detecting attacks.

* The main objective of this project is to develop a systematic approach to generate diverse and comprehensive benchmark dataset for intrusion detection based on the creation of user profiles which contain abstract representations of events and behaviours seen on the network.

In [24]:
#Let's load our data set
df=pd.read_csv("F:/upwork/Network anomaly detection/project/projectDataset.csv")

<IPython.core.display.Javascript object>

In [25]:
#Lets have a overlook at the data set
df.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,Bwd_Pkt_Len_Max,Bwd_Pkt_Len_Min,Bwd_Pkt_Len_Mean,Bwd_Pkt_Len_Std,Flow_Byts/s,Flow_Pkts/s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Tot,Fwd_IAT_Mean,Bwd_IAT_Mean,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Tot,Bwd_IAT_Mean.1,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Len,Bwd_Header_Len,Fwd_Pkts/s,Bwd_Pkts/s,Pkt_Len_Min,Pkt_Len_Max,Pkt_Len_Mean,Pkt_Len_Std,Pkt_Len_Var,FIN_Flag_Cnt,SYN_Flag_Cnt,RST_Flag_Cnt,PSH_Flag_Cnt,ACK_Flag_Cnt,URG_Flag_Cnt,CWE_Flag_Count,ECE_Flag_Cnt,Down/Up_Ratio,Pkt_Size_Avg,Fwd_Seg_Size_Avg,Bwd_Seg_Size_Avg,Fwd_Byts/b_Avg,Fwd_Pkts/b_Avg,Fwd_Blk_Rate_Avg,Bwd_Byts/b_Avg,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Subflow_Fwd_Pkts,Subflow_Fwd_Byts,Subflow_Bwd_Pkts,Subflow_Bwd_Byts,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25-07-2019 03:25,75,1,1,982,1430,982,982,982.0,0.000000,1430,1430,1430.000000,0.000000,3.216000e+07,26666.666670,75.0,0.000000,75,75,0,0.0,0.0,0,0,0,0.0,0.000000,0,0,0,0,0,0,8,8,13333.333330,13333.333330,982,1430,1280.666667,258.652921,66901.333330,0,0,0,0,0,0,0,0,1,1921.000000,982.0,1430.000000,0,0,0,0,0,0,1,982,1,1430,-1,-1,1,0,0.0,0.0,0,0,75.0,0.000000,75,75,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26-05-2019 22:11,5310,1,2,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,0.000000e+00,564.971751,2655.0,2261.327486,4254,1056,0,0.0,0.0,0,0,5310,5310.0,0.000000,5310,5310,0,0,0,0,20,44,188.323917,376.647834,0,0,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,2,0.000000,0.0,0.000000,0,0,0,0,0,0,1,0,2,0,-1,14600,0,0,0.0,0.0,0,0,2655.0,2261.327486,4254,1056,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11-07-2019 01:24,141,0,3,0,2806,0,0,0.0,0.000000,1388,30,935.333333,784.041666,1.990071e+07,21276.595740,70.5,0.707107,71,70,0,0.0,0.0,0,0,141,70.5,0.707107,71,70,0,0,0,0,0,96,0.000000,21276.595740,30,1388,1048.500000,679.000000,461041.000000,0,0,0,0,1,0,0,0,0,1398.000000,0.0,935.333333,0,0,0,0,0,0,0,0,3,2806,-1,1869,0,0,0.0,0.0,0,0,70.5,0.707107,71,70,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04-09-2019 03:58,151,0,2,0,2776,0,0,0.0,0.000000,1388,1388,1388.000000,0.000000,1.838411e+07,13245.033110,151.0,0.000000,151,151,0,0.0,0.0,0,0,151,151.0,0.000000,151,151,0,0,0,0,0,64,0.000000,13245.033110,1388,1388,1388.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,2082.000000,0.0,1388.000000,0,0,0,0,0,0,0,0,2,2776,-1,1869,0,0,0.0,0.0,0,0,151.0,0.000000,151,151,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10-09-2019 01:41,153,2,1,886,420,452,434,443.0,12.727922,420,420,420.000000,0.000000,8.535948e+06,19607.843140,76.5,0.707107,77,76,76,76.0,0.0,76,76,0,0.0,0.000000,0,0,0,0,0,0,16,8,13071.895420,6535.947712,420,452,431.500000,15.176737,230.333333,0,0,0,0,0,0,0,0,0,575.333333,443.0,420.000000,0,0,0,0,0,0,2,886,1,420,-1,-1,2,0,0.0,0.0,0,0,76.5,0.707107,77,76,Anomaly,Mirai,Mirai-Hostbruteforceg


In [26]:
#Lets have a overlook at the data set
df.tail()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,Bwd_Pkt_Len_Max,Bwd_Pkt_Len_Min,Bwd_Pkt_Len_Mean,Bwd_Pkt_Len_Std,Flow_Byts/s,Flow_Pkts/s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Tot,Fwd_IAT_Mean,Bwd_IAT_Mean,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Tot,Bwd_IAT_Mean.1,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Len,Bwd_Header_Len,Fwd_Pkts/s,Bwd_Pkts/s,Pkt_Len_Min,Pkt_Len_Max,Pkt_Len_Mean,Pkt_Len_Std,Pkt_Len_Var,FIN_Flag_Cnt,SYN_Flag_Cnt,RST_Flag_Cnt,PSH_Flag_Cnt,ACK_Flag_Cnt,URG_Flag_Cnt,CWE_Flag_Count,ECE_Flag_Cnt,Down/Up_Ratio,Pkt_Size_Avg,Fwd_Seg_Size_Avg,Bwd_Seg_Size_Avg,Fwd_Byts/b_Avg,Fwd_Pkts/b_Avg,Fwd_Blk_Rate_Avg,Bwd_Byts/b_Avg,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Subflow_Fwd_Pkts,Subflow_Fwd_Byts,Subflow_Bwd_Pkts,Subflow_Bwd_Byts,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
625778,192.168.0.24-210.89.164.90-56112-8043-17,192.168.0.24,56112,210.89.164.90,8043,17,25-07-2019 03:25,277,1,1,18,18,18,18,18.0,0.0,18,18,18.0,0.0,1.299639e+05,7220.216606,277.0,0.000000,277,277,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,8,8,3610.108303,3610.108303,18,18,18.0,0.0,0.0,0,0,0,0,0,0,0,0,1,27.000000,18.0,18.0,0,0,0,0,0,0,1,18,1,18,-1,-1,1,0,0.0,0.0,0,0,277.0,0.000000,277,277,Anomaly,Mirai,Mirai-UDP Flooding
625779,192.168.0.13-222.131.171.244-554-4570-6,222.131.171.244,4570,192.168.0.13,554,6,26-05-2019 22:06,1658,0,2,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000e+00,1206.272618,1658.0,0.000000,1658,1658,0,0.0,0.0,0,0,1658,1658.0,0.0,1658,1658,0,0,0,0,0,44,0.000000,1206.272618,0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0.000000,0.0,0.0,0,0,0,0,0,0,0,0,2,0,-1,14600,0,0,0.0,0.0,0,0,1658.0,0.000000,1658,1658,Anomaly,DoS,DoS-Synflooding
625780,192.168.0.13-192.168.0.16-9020-52739-6,192.168.0.16,52739,192.168.0.13,9020,6,11-07-2019 01:29,77,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000e+00,25974.025970,77.0,0.000000,77,77,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,12987.012990,12987.012990,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.000000,0.0,0.0,0,0,0,0,0,0,1,0,1,0,-1,32679,0,0,0.0,0.0,0,0,77.0,0.000000,77,77,Anomaly,Scan,Scan Port OS
625781,192.168.0.13-192.168.0.16-9020-49784-6,192.168.0.13,9020,192.168.0.16,49784,6,20-05-2019 05:00,240,2,1,2776,1388,1388,1388,1388.0,0.0,1388,1388,1388.0,0.0,1.735000e+07,12500.000000,120.0,7.071068,125,115,125,125.0,0.0,125,125,0,0.0,0.0,0,0,0,0,0,0,64,32,8333.333333,4166.666667,1388,1388,1388.0,0.0,0.0,0,0,0,0,1,0,0,0,0,1850.666667,1388.0,1388.0,0,0,0,0,0,0,2,2776,1,1388,-1,1869,2,0,0.0,0.0,0,0,120.0,7.071068,125,115,Normal,Normal,Normal
625782,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25-07-2019 03:25,198,2,1,2860,1430,1430,1430,1430.0,0.0,1430,1430,1430.0,0.0,2.170000e+07,15151.515150,99.0,32.526912,122,76,122,122.0,0.0,122,122,0,0.0,0.0,0,0,0,0,0,0,16,8,10101.010100,5050.505051,1430,1430,1430.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1906.666667,1430.0,1430.0,0,0,0,0,0,0,2,2860,1,1430,-1,-1,2,0,0.0,0.0,0,0,99.0,32.526912,122,76,Anomaly,Mirai,Mirai-UDP Flooding


In [27]:
df.shape

(625783, 86)

There are lots of columns, so let's look at them a few at a time

In [28]:
df.iloc[:,0:28].head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,Bwd_Pkt_Len_Max,Bwd_Pkt_Len_Min,Bwd_Pkt_Len_Mean,Bwd_Pkt_Len_Std,Flow_Byts/s,Flow_Pkts/s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Tot,Fwd_IAT_Mean
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25-07-2019 03:25,75,1,1,982,1430,982,982,982.0,0.000000,1430,1430,1430.000000,0.000000,3.216000e+07,26666.666670,75.0,0.000000,75,75,0,0.0
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26-05-2019 22:11,5310,1,2,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,0.000000e+00,564.971751,2655.0,2261.327486,4254,1056,0,0.0
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11-07-2019 01:24,141,0,3,0,2806,0,0,0.0,0.000000,1388,30,935.333333,784.041666,1.990071e+07,21276.595740,70.5,0.707107,71,70,0,0.0
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04-09-2019 03:58,151,0,2,0,2776,0,0,0.0,0.000000,1388,1388,1388.000000,0.000000,1.838411e+07,13245.033110,151.0,0.000000,151,151,0,0.0
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10-09-2019 01:41,153,2,1,886,420,452,434,443.0,12.727922,420,420,420.000000,0.000000,8.535948e+06,19607.843140,76.5,0.707107,77,76,76,76.0


In [29]:
df.iloc[:,28:56].head()

,Bwd_IAT_Mean,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Tot,Bwd_IAT_Mean.1,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Len,Bwd_Header_Len,Fwd_Pkts/s,Bwd_Pkts/s,Pkt_Len_Min,Pkt_Len_Max,Pkt_Len_Mean,Pkt_Len_Std,Pkt_Len_Var,FIN_Flag_Cnt,SYN_Flag_Cnt,RST_Flag_Cnt,PSH_Flag_Cnt,ACK_Flag_Cnt,URG_Flag_Cnt,CWE_Flag_Count
0,0.0,0,0,0,0.0,0.000000,0,0,0,0,0,0,8,8,13333.333330,13333.333330,982,1430,1280.666667,258.652921,66901.333330,0,0,0,0,0,0,0
1,0.0,0,0,5310,5310.0,0.000000,5310,5310,0,0,0,0,20,44,188.323917,376.647834,0,0,0.000000,0.000000,0.000000,0,1,0,0,0,0,0
2,0.0,0,0,141,70.5,0.707107,71,70,0,0,0,0,0,96,0.000000,21276.595740,30,1388,1048.500000,679.000000,461041.000000,0,0,0,0,1,0,0
3,0.0,0,0,151,151.0,0.000000,151,151,0,0,0,0,0,64,0.000000,13245.033110,1388,1388,1388.000000,0.000000,0.000000,0,0,0,0,1,0,0
4,0.0,76,76,0,0.0,0.000000,0,0,0,0,0,0,16,8,13071.895420,6535.947712,420,452,431.500000,15.176737,230.333333,0,0,0,0,0,0,0


In [30]:
df.iloc[:,56:86].head()

,ECE_Flag_Cnt,Down/Up_Ratio,Pkt_Size_Avg,Fwd_Seg_Size_Avg,Bwd_Seg_Size_Avg,Fwd_Byts/b_Avg,Fwd_Pkts/b_Avg,Fwd_Blk_Rate_Avg,Bwd_Byts/b_Avg,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Subflow_Fwd_Pkts,Subflow_Fwd_Byts,Subflow_Bwd_Pkts,Subflow_Bwd_Byts,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,0,1,1921.000000,982.0,1430.000000,0,0,0,0,0,0,1,982,1,1430,-1,-1,1,0,0.0,0.0,0,0,75.0,0.000000,75,75,Anomaly,Mirai,Mirai-Ackflooding
1,0,2,0.000000,0.0,0.000000,0,0,0,0,0,0,1,0,2,0,-1,14600,0,0,0.0,0.0,0,0,2655.0,2261.327486,4254,1056,Anomaly,DoS,DoS-Synflooding
2,0,0,1398.000000,0.0,935.333333,0,0,0,0,0,0,0,0,3,2806,-1,1869,0,0,0.0,0.0,0,0,70.5,0.707107,71,70,Anomaly,Scan,Scan Port OS
3,0,0,2082.000000,0.0,1388.000000,0,0,0,0,0,0,0,0,2,2776,-1,1869,0,0,0.0,0.0,0,0,151.0,0.000000,151,151,Anomaly,Mirai,Mirai-Hostbruteforceg
4,0,0,575.333333,443.0,420.000000,0,0,0,0,0,0,2,886,1,420,-1,-1,2,0,0.0,0.0,0,0,76.5,0.707107,77,76,Anomaly,Mirai,Mirai-Hostbruteforceg


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625783 entries, 0 to 625782
Data columns (total 86 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow_ID            625783 non-null  object 
 1   Src_IP             625783 non-null  object 
 2   Src_Port           625783 non-null  int64  
 3   Dst_IP             625783 non-null  object 
 4   Dst_Port           625783 non-null  int64  
 5   Protocol           625783 non-null  int64  
 6   Timestamp          625783 non-null  object 
 7   Flow_Duration      625783 non-null  int64  
 8   Tot_Fwd_Pkts       625783 non-null  int64  
 9   Tot_Bwd_Pkts       625783 non-null  int64  
 10  TotLen_Fwd_Pkts    625783 non-null  int64  
 11  TotLen_Bwd_Pkts    625783 non-null  int64  
 12  Fwd_Pkt_Len_Max    625783 non-null  int64  
 13  Fwd_Pkt_Len_Min    625783 non-null  int64  
 14  Fwd_Pkt_Len_Mean   625783 non-null  float64
 15  Fwd_Pkt_Len_Std    625783 non-null  float64
 16  Bw

In [32]:
df.describe()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,Bwd_Pkt_Len_Max,Bwd_Pkt_Len_Min,Bwd_Pkt_Len_Mean,Bwd_Pkt_Len_Std,Flow_Byts/s,Flow_Pkts/s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Tot,Fwd_IAT_Mean,Bwd_IAT_Mean,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Tot,Bwd_IAT_Mean.1,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Len,Bwd_Header_Len,Fwd_Pkts/s,Bwd_Pkts/s,Pkt_Len_Min,Pkt_Len_Max,Pkt_Len_Mean,Pkt_Len_Std,Pkt_Len_Var,FIN_Flag_Cnt,SYN_Flag_Cnt,RST_Flag_Cnt,PSH_Flag_Cnt,ACK_Flag_Cnt,URG_Flag_Cnt,CWE_Flag_Count,ECE_Flag_Cnt,Down/Up_Ratio,Pkt_Size_Avg,Fwd_Seg_Size_Avg,Bwd_Seg_Size_Avg,Fwd_Byts/b_Avg,Fwd_Pkts/b_Avg,Fwd_Blk_Rate_Avg,Bwd_Byts/b_Avg,Bwd_Pkts/b_Avg,Bwd_Blk_Rate_Avg,Subflow_Fwd_Pkts,Subflow_Fwd_Byts,Subflow_Bwd_Pkts,Subflow_Bwd_Byts,Init_Fwd_Win_Byts,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
count,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.00000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,6.257830e+05,6.257830e+05,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.0,625783.000000,625783.0,625783.000000,625783.000000,625783.000000,6.257830e+05,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.0,625783.0,625783.0,625783.0,625783.0,625783.0,625783.000000,625783.000000,625783.00000,625783.000000,625783.0,625783.000000,625783.000000,625783.0,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000,625783.000000
mean,35026.156190,16387.027479,9.971436,635.422865,1.675566,1.46853,570.738980,929.280973,392.489726,348.126571,373.556651,28.161094,681.424563,588.665338,637.107254,62.991166,inf,inf,483.462553,63.136879,565.820169,443.103039,102.157949,52.515505,27.689250,85.024112,34.679327,516.792743,446.964592,28.790664,486.704898,428.857417,0.0,0.026410,0.0,0.000051,22.490408,33.719612,5.030727e+04,27818.105271,511.645564,700.089229,633.666814,102.553417,69676.670743,0.000527,0.092946,0.000486,0.026410,0.530475,0.000051,0.000206,0.000067,0.364652,915.246266,373.556651,637.107254,0.0,0.0,0.0,0.0,0.0,0.0,1.675566,570.738980,1.46853,929.280973,-1.0,5880.924744,1.509913,0.0,3.764405,0.353385,4.248735,3.462159,502.503832,52.403995,561.540512,467.264459
std,24721.047752,17550.363037,5.379857,3496.740723,4.309970,1.21949,1161.873195,1731.760875,619.575865,588.161845,596.552835,144.356190,694.997621,682.938908,669.563565,227.773332,NaN,NaN,1892.861521,1159.713184,2866.325298,1719.332813,2216.304460,1212.002429,959.023292,1935.611451,1000.385742,3147.286789,2242.463582,808.672338,2751.967326,2177.830162,0.0,0.160352,0.0,0.007151,41.530486,40.283654,1.644118e+05,78481.469928,653.951208,696.842817,652.306047,243.227387,179138.678815,0.022958,0.290357,0.022035,0.160352,0.499071,0.007151,0.014356,0.008192,0.499153,948.679333,596.552835,669.563565,0.0,0.0,0.0,0.0,0.0,0.0,4.309970,1161.873195,1.21949,1731.760875,0.0,11529.622909,4.332737,0.0,68.064508,20.723370,88.934148,64.111043,2112.957360,1153.184897,2866.497606,1931.909971
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2.000540e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000

## Description of each variable
1. Src_Port--source port
2. Dst_IP--Destination IP address
3. Dst_port--Destination port
4. protocol-- a set of rules that dictate how data should be delivered over the public network (Internet).
5. Flow_Duration--Flow_Duration
6. Tot_Fwd_Pkts--Total packets in the forward direction
7. Tot_Bwd_Pkts--Total packets in the backward direction
8. TotLen_Fwd_Pkts--Total size of packet in forward direction
9. TotLen_Bwd_Pkts--Total size of packet in backward direction
10. Fwd_Pkt_Len_Max--Maximum size of packet in forward direction
11. Fwd_Pkt_Len_Min--Minimum size of packet in forward direction
12. Fwd_Pkt_Len_Mean--Average size of packet in forward direction
13. Fwd_Pkt_Len_Std--Standard deviation size of packet in forward direction
14. Bwd_Pkt_Len_Max--Maximum size of packet in backward direction
15. Bwd_Pkt_Len_Min--Minimum size of packet in backward direction
16. Bwd_Pkt_Len_Mean--Mean size of packet in backward direction
17. Bwd_Pkt_Len_Std--Standard deviation size of packet in backward direction
18. Flow_Byts/s--flow byte rate that is number of packets transferred per second
19. Flow_Pkts/s--flow packets rate that is number of packets transferred per second
20. Flow_IAT_Mean--Average time between two flows
21. Flow_IAT_Std--Standard deviation time two flows
22. Flow_IAT_Max--Maximum time between two flows
23. Flow_IAT_Min--Minimum time between two flows
24. Fwd_IAT_Tot--Total time between two packets sent in the forward direction
25. Fwd_IAT_Mean--Mean time between two packets sent in the forward direction
26. Fwd_IAT_Max--Maximum time between two packets sent in the forward direction
27. Fwd_IAT_Min--Minimum time between two packets sent in the forward direction
28. Bwd_IAT_Tot--Total time between two packets sent in the backward direction
29. Bwd_IAT_Mean.1--Mean time between two packets sent in the backward direction
30. Bwd_IAT_Mean--Mean time between two packets sent in the backward direction
31. Bwd_IAT_Std--Standard deviation time between two packets sent in the backward direction
32. Bwd_IAT_Max--Maximum time between two packets sent in the backward direction
33. Bwd_IAT_Min--Minimum time between two packets sent in the backward direction
34. Fwd_PSH_Flags--Number of times the PSH flag was set in packets travelling in the forward direction (0 for UDP)
35. Bwd_PSH_Flags--Number of times the PSH flag was set in packets travelling in the backward direction (0 for UDP)
36. Fwd_URG_Flags--	Number of times the URG flag was set in packets travelling in the forward direction (0 for UDP)
37. Bwd_URG_Flags--Number of times the URG flag was set in packets travelling in the backward direction (0 for UDP)
38. Fwd_Header_Len--Total bytes used for headers in the forward direction
39. Bwd_Header_Len--Total bytes used for headers in the forward direction
40. Fwd_Pkts/s--Number of forward packets per second
41. Bwd_Pkts/s--Number of backward packets per second
42. Pkt_Len_Min--Minimum length of a flow
43. Pkt_Len_Max--Maximum length of a flow
44. Pkt_Len_Mean--Mean length of a flow
45. Pkt_Len_Std--Standard deviation length of a flow
46. Pkt_Len_Var--Minimum inter-arrival time of packet
47. FIN_Flag_Cnt--Number of packets with FIN
48. SYN_Flag_Cnt--Number of packets with SYN
49. RST_Flag_Cnt--Number of packets with RST
50. PSH_Flag_Cnt--Number of packets with PUSH
51. ACK_Flag_Cnt--Number of packets with ACK(acknowledgement)
52. URG_Flag_Cnt--Number of packets with URG
53. CWE_Flag_Count--Number of packets with CWE
54. ECE_Flag_Cnt--Number of packets with ECE
55. Down/Up_Ratio--Download and upload ratio
56. Pkt_Size_Avg--Average size of packet
57. Fwd_Seg_Size_Avg--Average size observed in the forward direction
58. Bwd_Seg_Size_Avg--Average size observed in the backward direction
59. Fwd_Byts/b_Avg--Average number of bytes bulk rate in the forward direction
60. Fwd_Pkts/b_Avg--Average number of packets bulk rate in the forward direction
61. Fwd_Blk_Rate_Avg--Average number of bulk rate in the forward direction
62. Bwd_Byts/b_Avg--Average number of bytes bulk rate in the backward direction
63. Bwd_Pkts/b_Avg--Average number of packets bulk rate in the backward direction
64. Bwd_Blk_Rate_Avg--Average number of bulk rate in the backward direction
65. Subflow_Fwd_Pkts--The average number of packets in a sub flow in the forward direction
66. Subflow_Fwd_Byts--The average number of bytes in a sub flow in the forward direction
67. Subflow_Bwd_Pkts--The average number of packets in a sub flow in the backward direction
68. Subflow_Bwd_Byts--The average number of bytes in a sub flow in the backward direction
69. Init_Fwd_Win_Byts--Number of bytes sent in initial window in the forward direction
70. Init_Bwd_Win_Byts--number of bytes sent in initial window in the backward direction
71. Fwd_Act_Data_Pkts--number of packets with at least 1 byte of TCP data payload in the forward direction
72. Fwd_Seg_Size_Min--Minimum segment size observed in the forward direction
73. Active_Mean--Mean time a flow was active before becoming idle
74. Active_Std--Standard deviation time a flow was active before becoming idle
75. Active_Max--Maximum time a flow was active before becoming idle
76. Active_Min--Minimum time a flow was active before becoming idle
77. Idle_Mean--Mean time a flow was idle before becoming active
78. Idle_Std--Standard deviation time a flow was idle before becoming active
79. Idle_Max--Maximum time a flow was idle before becoming active
80. Idle_Min--Minimum time a flow was idle before becoming active

#### Lets check for the unique and null values

In [33]:
df.isnull().sum().sort_values(ascending=False)

Sub_Cat             0
Bwd_IAT_Tot         0
Flow_IAT_Std        0
Flow_IAT_Max        0
Flow_IAT_Min        0
                   ..
Pkt_Size_Avg        0
Fwd_Seg_Size_Avg    0
Bwd_Seg_Size_Avg    0
Fwd_Byts/b_Avg      0
Flow_ID             0
Length: 86, dtype: int64

In [34]:
# Lets examine or target variable
df.Label.isnull().sum()

0

*  There are no missing values in the response variable column, which is good

In [35]:
df.Label.value_counts()

Anomaly    585710
Normal      40073
Name: Label, dtype: int64

In [36]:
df.Cat.value_counts()

Mirai                415677
Scan                  75265
DoS                   59391
Normal                40073
MITM ARP Spoofing     35377
Name: Cat, dtype: int64

#### Let's look at the number of unique values in each of the categoric columns

In [37]:
df.select_dtypes(include = 'object').nunique().sort_values(ascending = False)

Flow_ID      64064
Src_IP       57985
Dst_IP         478
Timestamp      110
Sub_Cat          9
Cat              5
Label            2
dtype: int64

- Since the `Flow_ID`,`Src_IP`,`Dst_IP`,`Timestamp`,`Cat`,`Sub_Cat` will not play important role so lets drop them

In [38]:
df = df.drop(['Flow_ID','Src_IP','Dst_IP','Timestamp','Cat','Sub_Cat'], axis = 1)

In [40]:
df.shape[1]

80

We're left with 80 columns including the Target variable

### Since our data is clean there are no missing values lets further analyse the Each variable 1 by 1

In [44]:
df.nunique().sort_values(ascending=True)

Init_Fwd_Win_Byts        1
Bwd_Pkts/b_Avg           1
Bwd_Blk_Rate_Avg         1
Bwd_Byts/b_Avg           1
Fwd_Blk_Rate_Avg         1
                     ...  
Flow_IAT_Mean        19053
Flow_Pkts/s          19578
Idle_Std             22807
Flow_IAT_Std         28879
Flow_Byts/s          30354
Length: 80, dtype: int64

**We observed that some columns are having only 1 value. Further we will check these values and we will drop these columns as single values are of no use when we train our model**

- Fwd_PSH_Flags--Number of times the PSH flag was set in packets travelling in the forward direction (0 for UDP)

In [74]:
df.Fwd_PSH_Flags.unique().tolist()

[0]

- Fwd_URG_Flags--	Number of times the URG flag was set in packets travelling in the forward direction (0 for UDP)

In [75]:
df.Fwd_URG_Flags.unique().tolist()

[0]

- Fwd_Byts/b_Avg--Average number of bytes bulk rate in the forward direction

In [77]:
df['Fwd_Byts/b_Avg'].unique().tolist()

[0]

- Fwd_Pkts/b_Avg --Average number of bytes bulk rate in the forward direction

In [79]:
df['Fwd_Pkts/b_Avg'].unique().tolist()

[0]

- Fwd_Blk_Rate_Avg--Average number of bulk rate in the forward direction

In [80]:
df.Fwd_Blk_Rate_Avg.unique().tolist()

[0]

- Bwd_Byts/b_Avg----Average number of bytes bulk rate in the backward direction

In [81]:
df['Bwd_Byts/b_Avg'].unique().tolist()

[0]

- Bwd_Pkts/b_Avg----Average number of packets bulk rate in the backward direction

In [82]:
df['Bwd_Pkts/b_Avg'].unique().tolist()

[0]

- Bwd_Blk_Rate_Avg--Average number of bulk rate in the backward direction

In [83]:
df.Bwd_Blk_Rate_Avg.unique().tolist()

[0]

- Init_Fwd_Win_Byts----Number of bytes sent in initial window in the forward direction

In [84]:
df.Init_Fwd_Win_Byts.unique().tolist()

[-1]

- Fwd_Seg_Size_Min----Minimum segment size observed in the forward direction

In [85]:
df.Fwd_Seg_Size_Min.unique().tolist()

[0]

### Findings:
`Fwd_PSH_Flags`,`Fwd_URG_Flags`,`Fwd_Byts/b_Avg`,`Fwd_Pkts/b_Avg`,`Fwd_Blk_Rate_Avg`,`Bwd_Byts/b_Avg`,`Bwd_Pkts/b_Avg`,`Bwd_Blk_Rate_Avg`,`Init_Fwd_Win_Byts`,`Fwd_Seg_Size_Min`
** These variables are having only single values and later we will drop these columns as these columns will not contribute while we build the model**

## Visualisation